In [74]:
import pandas as pd
from datetime import date, datetime
import re
import os
from imblearn.under_sampling import RandomUnderSampler
#Carregando arquivo excel em formato dataframe
dataframe_original = pd.read_excel('Dados_Tratados/Dados_Tratados_Amostra_02-10-25.xlsx', engine='openpyxl')
dataframe_teste = pd.read_excel('Dados_Tratados/Dados_Tratados_Teste_02-10-25.xlsx')
dataframe_teste_ajustado = pd.read_excel('Dados_Tratados/Dados_Tratados_Teste_Ajustado_02-10-25.xlsx')

#dataframe_original.head(5)

In [75]:
X = dataframe_original.drop("VALIDACAO", axis=1)  # features
y = dataframe_original["VALIDACAO"]   

ros = RandomUnderSampler(random_state=42)

X_res, y_res = ros.fit_resample(X, y)

df_balanceado = pd.concat([X_res, y_res], axis=1)

df_balanceado['VALIDACAO'].value_counts()

VALIDACAO
N    601
S    601
Name: count, dtype: int64

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

x = df_balanceado['DS_CONTEXTO']
y = df_balanceado['VALIDACAO']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.6, random_state=22, train_size=0.4)

pipeSVC = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf', LinearSVC())
])

pipeCNB = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf', ComplementNB())
])

pipeSVC.fit(x_train, y_train)
predictSVC = pipeSVC.predict(x_test)
accuracy_score(y_test,predictSVC)
print(f'SVC: {accuracy_score(y_test,predictSVC):.2f}')

pipeCNB.fit(x_train, y_train)
predictCNB = pipeCNB.predict(x_test)
accuracy_score(y_test,predictCNB)
print(f'CNB: {accuracy_score(y_test,predictCNB):.2f}')

SVC: 0.96
CNB: 0.94


In [86]:
dados_teste = dataframe_teste['DS_CONTEXTO']

predictSVC = pipeSVC.predict(dados_teste)

predictCNB = pipeCNB.predict(dados_teste)

In [87]:
df_classificado = pd.DataFrame({
    'DS_EVOLUCAO': dataframe_teste['DS_EVOLUCAO'],
    'DS_CONTEXTO': dados_teste,
    'VALIDACAO': dataframe_teste['VALIDACAO'],
    'PREDICAO_SVC': predictSVC,
    'PREDICAO_CNB': predictCNB
})

In [88]:
print(f"Dados Validacao: {df_classificado['VALIDACAO'].value_counts()}")
print(f"Validacao SVC: {df_classificado['PREDICAO_SVC'].value_counts()}")
print(f"Validacao CNB: {df_classificado['PREDICAO_CNB'].value_counts()}")

Dados Validacao: VALIDACAO
N    796
S    230
Name: count, dtype: int64
Validacao SVC: PREDICAO_SVC
N    752
S    274
Name: count, dtype: int64
Validacao CNB: PREDICAO_CNB
N    709
S    317
Name: count, dtype: int64


In [89]:
path = os.path.join('Resultados/')
file_name = 'Dados_Resultado_' + f"{datetime.today().strftime('%d-%m-%y')}" + ".xlsx"
file_path = path + file_name
df_classificado.to_excel(file_path,index=False)

In [90]:
dados_teste_ajustado = dataframe_teste_ajustado['DS_CONTEXTO']

predictSVC_ajustado = pipeSVC.predict(dados_teste_ajustado)

predictCNB_ajustado = pipeCNB.predict(dados_teste_ajustado)

In [91]:
df_classificado_ajustado = pd.DataFrame({
    'DS_EVOLUCAO': dataframe_teste_ajustado['DS_EVOLUCAO'],
    'DS_CONTEXTO': dados_teste_ajustado,
    'VALIDACAO': dataframe_teste_ajustado['VALIDACAO'],
    'PREDICAO_SVC': predictSVC_ajustado,
    'PREDICAO_CNB': predictCNB_ajustado
})

In [92]:
print(f"Dados Validacao: {df_classificado_ajustado['VALIDACAO'].value_counts()}")
print(f"Validacao SVC: {df_classificado_ajustado['PREDICAO_SVC'].value_counts()}")
print(f"Validacao CNB: {df_classificado_ajustado['PREDICAO_CNB'].value_counts()}")

Dados Validacao: VALIDACAO
N    714
S    312
Name: count, dtype: int64
Validacao SVC: PREDICAO_SVC
N    752
S    274
Name: count, dtype: int64
Validacao CNB: PREDICAO_CNB
N    709
S    317
Name: count, dtype: int64


In [93]:
path = os.path.join('Resultados/')
file_name = 'Dados_Resultado_Ajustado' + f"{datetime.today().strftime('%d-%m-%y')}" + ".xlsx"
file_path = path + file_name
df_classificado_ajustado.to_excel(file_path,index=False)